# In this notebook we compare the performance of the Sinkhorn and the damped Newton algorithm.

In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import scipy as scp
import pylab as pyl

import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)

%matplotlib inline
%load_ext autoreload
%autoreload 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
relative_path_to_new_folder = "../Images"
os.makedirs(relative_path_to_new_folder, exist_ok = True)
if not os.path.isdir('../Images/SinkhornvsDampedNewton_images'):
    os.makedirs('../Images/SinkhornvsDampedNewton_images')

In [ ]:
"""To compute distance matrix"""
def distmat( x, y ):
    return np.sum( x**2, 0 )[:,None] + np.sum( y**2, 0 )[None,:] - 2 * x.transpose().dot( y )

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP( u, K, v ):
    return u[:,None] * K *v[None,:]

def plotp( x, col, plt, scale = 200, edgecolors = "k"):
  return plt.scatter( x[0,:], x[1,:], s = scale, edgecolors = edgecolors,  c = col, cmap = 'plasma', linewidths = 2 )

In [ ]:
import computational_OT

## Entropy regularized formulation

The primal entropy regularized formulation of OT is given by:
$$
OT_{\varepsilon}(\alpha,\beta) = min_{\pi \in \mathcal{U}(\alpha,\beta)} \langle C,\pi \rangle +\varepsilon KL(\pi\|\alpha \otimes \beta)\ ,
$$
where
$\ 
KL(\pi\|\alpha \otimes \beta) 
\ $ is the KL-divergence and $\ \mathcal{U}(\alpha,\beta)=\{\pi: \pi\mathcal{1}=\alpha, \pi^{T}\mathcal{1}=\beta\}$.

# Sinkhorn iteration function
The optimal coupling $\pi^{*}$ has the following form :
$$
\pi^{*} = \alpha \odot diag(u)K diag(v)\odot \beta
$$
and we know that $\pi^{*}\mathbb{1}=\alpha$ and $(\pi^{*})^{T}\mathbb{1}=\beta$.
###
Therefore, Sinkhorn updates is given by the following alternative projections
$$
u^{t+1}  \leftarrow \frac{1}{K(v^{t}\odot \beta)}\ , \\
v^{t+1}  \leftarrow \frac{1}{K^{T}(u^{t+1}\odot \alpha)}\ , 
$$
where 
$K = e^{-\frac{C}{\varepsilon}}\in M_{n\times m}(\mathbb{R}),\ \alpha \in \mathbb{R}^{n},\ \beta \in \mathbb{R}^{m}\ ,\ u\in \mathbb{R}^{n},\ v\in \mathbb{R}^{m}\ and \ (u^{0},v^{0})=(u,v)\ .$


In [ ]:
def sinkhorn( epsilons, N, x, y, iterations = 1000 ):
    # Sinkhorn
    print("Sinkhorn.... ")
    SinkhornP                  = []
    results_Sinkhorn           = []
    times_Sinkhorn             = []
    Pmatrix_dist_linVSsinkhorn = []
    for eps in epsilons:

        print( "Sinkhorn for epsilon = "+str(eps)+":" )    
        #Cost matrix
        C = distmat( x, y )
        
        # a and b
        a = normalize( np.ones( N[0] ) )
        b = normalize( np.ones( N[1] ) )

        #Kernel
        K = np.exp( - C/eps )


        print("Doing for (",N[0],N[1],").")
        print( " |- Iterating")

        #Inflating
        u = a
        v = b

        start = time.time()
        Optimizer = computational_OT.Sinkhorn( K, a, b, u, v, eps )
        out       = Optimizer._update( maxiter = iterations )
        results_Sinkhorn.append( out )
        end = time.time()
        times_Sinkhorn.append( 1e3 * ( end - start ) )
        print( " |- Computing P")
        print( "" )
        SinkhornP.append( GetP( np.exp( out['potential_f']/eps ), K, np.exp( out['potential_g']/eps ) ) )
    return {
        'results_list': results_Sinkhorn,
        'time_stamps' : times_Sinkhorn,
        'Ps'          : SinkhornP
    }
    

# Damped Newton(without precondition) iteration function

The dual formulation of OT is given by
$$
OT_{\varepsilon} = \max_{f\in \mathbb{R}^{n}, g\in\mathbb{R}^{m}} \langle f, \alpha \rangle + \langle g, \beta \rangle - \varepsilon\left(\langle\alpha \otimes \beta, e^{\frac{f}{\varepsilon}}\odot K \odot e^{\frac{g}{\varepsilon}}  \rangle-1\right)\ ,
$$
where
$$
\alpha \in \mathcal{M}_{1}(\mathcal{X}),\ \beta \in \mathcal{M}_{1}(\mathcal{Y}),\ \varepsilon>0,\ f\in\mathbb{R}^{n},\ g\in \mathbb{R}^{m}\ .
$$
### 
The Hessian is given by 
$\nabla^{2}Q_{\varepsilon}(f,g)=\frac{-1}{\varepsilon}
\begin{pmatrix}
\Delta(\alpha) && \pi_{\varepsilon}\\
\pi^{T}_{\varepsilon} && \Delta(\beta) 
\end{pmatrix}
\ , \ $ where $\pi\mathbb{1}_{m} = \alpha,\ \pi^{T}\mathbb{1}_{n}=\beta,\ $ and $\Delta = diag: \mathbb{R}^{n} \rightarrow M_{n}(\mathbb{R})$ is the linear operator mapping a vector  to a diagonal matrix  containing  this vector.
#####

This implies 
$$
\begin{pmatrix}
\Delta(\alpha) && \pi_{\varepsilon}\\
\pi^{T}_{\varepsilon} && \Delta(\beta) 
\end{pmatrix}
\begin{pmatrix}
\mathbb{1}_{n}\\
\mathbb{1}_{m}
\end{pmatrix} = 0\ ,
$$
that is,
$$
\begin{pmatrix}
\mathbb{1}_{n}\\
\mathbb{1}_{m}
\end{pmatrix}\in Ker(\nabla^{2}Q_{\varepsilon}(f,g))\ .
$$
Hence, $\nabla^{2}Q_{\varepsilon}(f,g)$ is singular. Therefore, on regularization we have the following Hessian
$
H_{reg} := \nabla^{2}Q_{\varepsilon}(f,g)+\lambda cc^{T}\ ,
$ 
where $c= \begin{pmatrix}\frac{\mathbb{1}}{\sqrt{n+m}}\\-\frac{\mathbb{1}}{\sqrt{n+m}}\end{pmatrix}\in M_{(n+m),1}(\mathbb{R})$.
###
Now, at the $k^{th}$ iteration solve
$\nabla^{2}Q_{\varepsilon}(f,g)p_{k} = \nabla Q_{\varepsilon}(f,g)$ to obtain the optimizing direction vector $p_{k}$ and then perform the Armijo condition to obtain the update step $\alpha_{k}$ such that we have the update
$$
(f,g) \leftarrow (f,g) + \alpha_{k} p_{k}\ .
$$






In [ ]:
def dampednewton( epsilons, N, x, y, rho = 0.95, c = 0.05, iterations = 50 ):
    DampedNewtonP=[]
    results_DampedNewton  = []
    times_DampedNewton    = []
    Hessians_DampedNewton = []
    #Cost matrix
    C = distmat( x, y )

    # a and b
    a = normalize( np.ones( N[0] ) )
    b = normalize( np.ones( N[1] ) )

    for eps in epsilons:
        print("Damped Newton for epsilon = "+str(eps)+":")     

        #Kernel
        K = np.exp( - C/eps )
        f, g = a, b

        print("Doing for (",N[0], N[1],").")
        print( " |- Iterating")  
        start = time.time()
        Optimizer = computational_OT.DampedNewton( K, a, b, f, g, eps, rho, c )
        out = Optimizer._update( maxiter = iterations )
        results_DampedNewton.append( out )
        end = time.time()
        times_DampedNewton.append( 1e3 * ( end - start ) )
        print( " |- Computing P")
        DampedNewtonP.append( GetP( np.exp( out['potential_f']/eps ), K, np.exp( out['potential_g']/eps ) ) )
        print( " |- Recording (unstabilized) Hessian \n")

        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( np.concatenate( ( a, b ), axis = None ) )
        mat = diag[:,None] * mat * diag[None,:]
        Hessians_DampedNewton.append( mat )

    return {
        'results_list': results_DampedNewton,
        'time_stamps' : times_DampedNewton,
        'Ps'          : DampedNewtonP,
        'Hessians'    : Hessians_DampedNewton 
    }
    

# Damped Newton with preconditioning iteration function
Here we proceed similar to the damped Newton algorithm but with preconditioning.  We consider $t$ eigenvalues of the Hessian and form the following preconditioning matrix:
$$
P = \left(I_{n+m}-\sum_{i-1}^{t}\left(1 - \frac{1}{\sqrt{\lambda_{i}}}\right)y_{i}y_{i}^{T}\right)\ .
$$
Now, at the $k^{th}$ iteration we solve the following equation:
$$
(P\nabla^{2}Q_{\varepsilon}(f,g)P)(Pp_{k})=P\nabla Q_{\varepsilon}(f,g)\ ,
$$
using iterative inversion methods such as "Conjugate gradient" and "GMRES" to get the update direction $p_{k}$, following which we use the Armijo condition to obtain the step size $\alpha_{k}$.

In [ ]:
def dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, rho = 0.95, c = 0.05,  iterations = 500, iter_inv = 30, optimizer = 'cg', version = None, debug = False):
    reset_starting_point = True
    final_modified_Hessians = []
    DampedNewtonwithprecondP = []
    results_DampedNewtonwithprecond  = []
    times_DampedNewtonwithprecond    = []
    # Cost matrix
        
    C = distmat( x, y )

    # a and b
    a = normalize( np.ones(N[0]) )
    b = normalize( np.ones(N[1]) )

    f, g = None, None
    for eps in epsilons:
        print( "Damped Newton with preconditioning for epsilon = "+str(eps)+":" )    

        #Kernel
        K = np.exp( - C/eps )

        if (f is None) or (g is None): 
            f, g = a, b

        print( "Doing for (",N[0], N[1],")." )
        
        print( " |- Iterating" )  
        start = time.time()
        Optimizer = computational_OT.DampedNewton_With_Preconditioner( K, a, b, f, g, eps, rho, c, null_vector, precond_vectors[:] )
        out = Optimizer._update( maxiter = iterations, iterative_inversion = iter_inv, version = version, debug = debug, optType = optimizer )
        results_DampedNewtonwithprecond.append( out )
        end = time.time()
        
        times_DampedNewtonwithprecond.append( 1e3 * ( end - start ) )
        print( " |- Computing P" )

        if not reset_starting_point:
            f = Optimizer.x[:a.shape[0]]
            g = Optimizer.x[a.shape[0]:]
 

        DampedNewtonwithprecondP.append( GetP( np.exp( out['potential_f']/eps ), K, np.exp( out['potential_g']/eps ) ) )
        final_modified_Hessians.append( Optimizer.modified_Hessian )
    return {
        'results_list': results_DampedNewtonwithprecond,
        'time_stamps' : times_DampedNewtonwithprecond,
        'Ps'          : DampedNewtonwithprecondP,
        'Hessians'    : final_modified_Hessians 
    }
    

# Comparison for Data size 400 and 500

In [ ]:
N = [ 400, 500 ]

In [ ]:
x     = np.random.rand( 2, N[0] ) - 0.5
theta = 2 * np.pi * np.random.rand( 1, N[1] )
r     = 0.8 + .2 * np.random.rand( 1, N[1] )
y     = np.vstack( ( r * np.cos(theta), r * np.sin(theta) ) )

## I. Sinkhorn

In [ ]:
epsilons = [  0.03, 0.001, 0.0009, 0.00084 ]
results_sinkhorn = sinkhorn( epsilons, N, x, y, iterations = 20000 )

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )

plt.subplot( 2, 1, 1 ),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range(len(results_sinkhorn['results_list'])):
  error = np.asarray( results_sinkhorn['results_list'][i]['error_a'] ) + np.asarray( results_sinkhorn['results_list'][i]['error_b'] ) 
  plt.plot( error, label = 'Sinkhorn for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
plt.yscale( 'log' )
plt.legend()
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ConvergenceSinkhornvaryingepsilon.png" )
plt.show()

## II. Damped Newton without Preconditioning

In [ ]:
epsilons = [  0.5, 0.03, 0.02 ]
results_DampedNewton = dampednewton( epsilons, N, x, y )

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range(len(results_DampedNewton['results_list'])):
  error = np.asarray( results_DampedNewton['results_list'][i]['error_a'] ) + np.asarray( results_DampedNewton['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )

plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.savefig("../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithoutPrecond.png" )
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" )

### Objective function

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "Objective Function" )

for i in range(len(results_DampedNewton['results_list'])):
  plt.plot( np.asarray( results_DampedNewton['results_list'][i]['objectives'] ), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )

plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.legend()
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ObjectiveDampedNewtonwithoutPrecond.png" )
plt.show()


### Alpha plot


In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Alpha" )

for i in range(len(results_DampedNewton['results_list'])):
  plt.plot( np.asarray( results_DampedNewton['results_list'][i]['linesearch_steps'] ), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )

plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend()
plt.savefig( "../Images/SinkhornvsDampedNewton_images/AlphaDampedNewtonwithoutPrecond.png" )
plt.show()

### Plotting spectrum as a function of $\varepsilon$

In [ ]:
def spectral_decomposition( mat ):
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort( eig )
    eig = eig[ sorting_indices ]
    v   = v[ : , sorting_indices ]
    
    print( "List of smallest eigenvalues: ", eig[ : 10 ] )
    print( "List of largest  eigenvalues: ", eig[ - 10 : ] )
    return eig, v

In [ ]:
eigs = []
eigvecs = []
for i in range(len(epsilons)):
    eps = epsilons[i]
    print("Spectral statistics of Hessian for epsilon = "+str(eps))
    ev = spectral_decomposition( results_DampedNewton['Hessians'][i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")


In [ ]:
fig, ax=plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title("Histogram of eigenvalues.")
for i in range(len(epsilons)):
    ax[i].hist( eigs[i], 50)
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]))
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
plt.subplots_adjust( wspace = 0, hspace = 0.5 )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/eigenhistunstabilizedDampedNewtonwithoutPrecond.png" )
plt.show()

### Actual preconditioning

In [ ]:
def build_preconditioners( num_eigs, modified_Hessian, ansatz = True ):
    # Diagonalize
    eigenvalues, eigenvectors = np.linalg.eigh( modified_Hessian )
    sorting_indices = np.argsort( eigenvalues )
    eigenvalues  = eigenvalues[ sorting_indices ]
    eigenvectors = eigenvectors[ : , sorting_indices ]
    # Form null vector
    if not ansatz:
        null_vector = eigenvectors[ : , 0 ]
    else:
        null_vector = np.hstack( ( np.ones(N[0]), - np.ones(N[1]) ) )
        norm = np.sqrt( N[0] + N[1] )
        null_vector = null_vector/norm
    # Form other vectors (only 13)
    _, m = eigenvectors.shape
    indices = []
    for i in range(num_eigs//2):
        indices.append( m - i - 2 )
        indices.append( i + 1 )
    if num_eigs%2 != 0:
        indices.append( m - 1 - ( num_eigs//2 ) )
   
    precond_vectors = eigenvectors[ : , indices ]
    precond_vectors = []
    for index in indices:
        precond_vectors.append( eigenvectors[:,index] )
    #
    return null_vector, precond_vectors

In [ ]:
num_eigs = 13
null_vector, precond_vectors = build_preconditioners( num_eigs, results_DampedNewton['Hessians'][-1], ansatz = False )

## III. Damped Newton with Preconditioning

### Exact inversion

In [ ]:
epsilons = [ 0.5, 0.001 , 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 50, version = None, iter_inv = -1, rho = 0.95 )

#### Error plot

In [ ]:
plt.figure( figsize = ( 20,7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
  
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho.png" )
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" )

#### Spectrum of the Hessian

In [ ]:
eigs = []
eigvecs = []
for i in range(len(epsilons)):
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    ev = spectral_decomposition( results_DampedNewtonwithprecond['Hessians'][i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")


In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 3 ), nrows = 1, ncols = len(epsilons), sharey = True )
plt.title("Histogram of eigenvalues.")
for i in range(len(epsilons)):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]))
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
plt.subplots_adjust( wspace = 0, hspace = 0 )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/eigenhistunstabilizedDampedNewtonwithPrecond.png" )
plt.show()

### Iterative inversion = 2

#### rho = 0.5

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.001, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 2, rho = 0.5 )

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
  
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_5itv2.png" )
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" )

#### rho = 0.7

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 2, rho = 0.7 )

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
  
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_7itv2.png" )
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" )

#### rho = 0.9

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 2, rho = 0.9 )

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
  
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' ) 
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_9itv2.png" )
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" )

### Iterative Inversion = 5

#### rho = 0.5

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 5, rho = 0.5 )

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
  
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_5itv5.png" )
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" )

#### rho = 0.7

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 5, rho = 0.7 )

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
  
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_7itv5.png" )
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" )

#### rho = 0.9

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 5, rho = 0.9 )

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
  
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_9itv5.png" )
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" )

### Iterative Inversion = 10

#### rho = 0.5

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 10, rho = 0.5 )

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
  
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_5itv10.png" ) 
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" )

#### rho = 0.7

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 10, rho = 0.7 )

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error = np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_a']) + np.asarray(results_DampedNewtonwithprecond['results_list'][i]['error_b'])
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
  
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_7itv10.png" )
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" )

#### rho = 0.9

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition( epsilons, N, x, y, null_vector, precond_vectors, iterations = 500,iter_inv = 10, rho = 0.9 )

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
  
plt.xlabel( "Number of iterations" ) 
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_9itv10.png" )
plt.show()

print( "\n Error plots can increase! The error is not the objective function!" ) 

#### rho = 0.95

In [ ]:
epsilons = [ 0.5, 0.03, 0.02, 0.00084 ] 
results_DampedNewtonwithprecond = dampednewtonprecondition(  epsilons, N, x, y, null_vector, precond_vectors, iterations = 500, iter_inv = 10, rho = 0.95 )

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range(len(results_DampedNewtonwithprecond['results_list'])):
  error = np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_a'] ) + np.asarray( results_DampedNewtonwithprecond['results_list'][i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
  
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()  
plt.yscale( 'log' )
plt.savefig( "../Images/SinkhornvsDampedNewton_images/ErrorDampedNewtonwithPrecondrho0_95itv10.png" )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )